In [303]:
import cv2
from paddleocr import PPStructure
from bs4 import BeautifulSoup
import re

In [304]:
img_path= "image/IMG-20240528-WA0020.jpg"

table_engine = PPStructure(layout=False, show_log=True, ocr=True)
img = cv2.imread(img_path)
result = table_engine(img)

# result[0]['res']['html']

[2024/06/09 18:37:54] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\ASUS/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\ASUS/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_lengt

In [305]:
# convert html to text pake beautifulsoup
soup = BeautifulSoup(result[0]['res']['html'], 'html.parser')
td_elements = soup.find_all('td')

extracted_text = [td.get_text(strip=True) for td in td_elements]

text = "\n".join(extracted_text).lower()
print(text)


bpom ri #
一

informasi nilai gizi
nutrition facts
takarani saj/serving size : 16g 2 sajan per kemasan/ 2 serving per packing
jumlah per sajian/ amount per serving
energitotal/total energy:80kkal energi dari lemak/
energyfromfat:35kkal
%akg* lemak total /total fat 3.5g 6%
lemak jenuh/ saturated fat 3.5g17%
protein/ protein 1g2% karbohidrat total/ total carbohydrate 11g 3%
serat pangan/
dietary fiber 1g3% gula/sugar 5g
garam (natrium)/ 30mg 2%
salt (sodium) *persen akg berdasarkan kebutuhan
energi2150 kkal.kebutuhan energi anda mungkin lebih tinggi atau lebih
rendah. percent daily values are based on a
2150kcaldtourdalyvaluemay be higher or lower depending on your
calorie needs.

8

o



In [306]:
def cleaning_text(text):
    # buat new line setiap ada '%' 
    text = re.sub(r'%', '%\n', text)
    #replace '9' with 'g' kalo setelahnya space atau end of line
    text = re.sub(r'(?<=\d)9(?=\s|$)', 'g', text)
    return text

In [307]:
cleaned_text = cleaning_text(text)
print(cleaned_text)

bpom ri #
一

informasi nilai gizi
nutrition facts
takarani saj/serving size : 16g 2 sajan per kemasan/ 2 serving per packing
jumlah per sajian/ amount per serving
energitotal/total energy:80kkal energi dari lemak/
energyfromfat:35kkal
%
akg* lemak total /total fat 3.5g 6%

lemak jenuh/ saturated fat 3.5g17%

protein/ protein 1g2%
 karbohidrat total/ total carbohydrate 11g 3%

serat pangan/
dietary fiber 1g3%
 gula/sugar 5g
garam (natrium)/ 30mg 2%

salt (sodium) *persen akg berdasarkan kebutuhan
energi2150 kkal.kebutuhan energi anda mungkin lebih tinggi atau lebih
rendah. percent daily values are based on a
2150kcaldtourdalyvaluemay be higher or lower depending on your
calorie needs.

8

o



In [308]:
def find_value(text,  keyword):
    lines = text.split('\n')
    for i, line in enumerate(lines):
        if keyword in line.lower():
            pattern = rf"({keyword}.*?)(\d+(?:\.\d+)?)"
            match = re.search(pattern, text, re.DOTALL)
            if match:
                return float(match.group(2))
    return float(0)

In [309]:
sajian = find_value(text, 'serving')
energy_value = find_value(cleaned_text, 'energi')
protein_value = find_value(cleaned_text, 'protein')
lemak_value = find_value(cleaned_text, 'lemak total')
karbohidrat_value = find_value(cleaned_text, 'karbohidrat')
serat_value = find_value(cleaned_text, 'serat')
natrium_value = find_value(cleaned_text, 'natrium')

print('Jumlah Per Sajian :', sajian, "\nEnergi :", energy_value, "\n""Protein :", protein_value, "\n""Lemak :", lemak_value, "\n""Karbohidrat :", karbohidrat_value, "\n""Serat :", serat_value, "\n""Natrium :", natrium_value)

# print("Energi :", energy_value, "\n""Protein :", protein_value, "\n""Lemak :", lemak_value, "\n""Karbohidrat :", karbohidrat_value, "\n""Serat :", serat_value, "\n""Natrium :", natrium_value)

Jumlah Per Sajian : 16.0 
Energi : 80.0 
Protein : 1.0 
Lemak : 3.5 
Karbohidrat : 11.0 
Serat : 1.0 
Natrium : 30.0
